# Notebook for exploring different models and prompts

## Setup

### Do Imports

In [ ]:
# SQLite compatibility fix for Streamlit Cloud
import sys
sys.path.append('..') 

try:
    __import__('pysqlite3')
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
except ImportError:
    pass

from src.rag_system import RAGSystem
from src.document_processor import DocumentProcessor
from src.vector_store import VectorStore

### Go here to figure out which models to test

In [ ]:
import os
from openai import OpenAI

# HuggingFace API configuration
api_token = os.getenv("HUGGINGFACE_API_TOKEN")

# Initialize OpenAI client with HuggingFace router
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=api_token
)

models = client.models.list()
for model in models:
    #if 'gpt' in model.id.lower():
    print(model.id)
        
        
    

### Selected Models

I hand selected a few models, and then below ran a cost analysis to make sure I wasn't getting into anything too expensive.

In [ ]:
selected_models = [
    'openai/gpt-oss-20b',
    'deepseek-ai/DeepSeek-V3-0324',
    'google/gemma-3-27b-it',
    'meta-llama/Llama-Guard-4-12B'
]

models_json = [model.model_dump() for model in client.models.list()]

for model in models_json:
    if model['id'] in selected_models:
        lowest_price = None
        lowest_provider = None
        for provider in model.get('providers', []):
            pricing = provider.get('pricing', {})
            # Check both input and output prices if available
            input_price = pricing.get('input')
            output_price = pricing.get('output')
            for price in [input_price, output_price]:
                if price is not None:
                    if lowest_price is None or price < lowest_price:
                        lowest_price = price
                        lowest_provider = provider.get('provider')
        if lowest_price is not None:
            print(f"{model['id']}: {lowest_provider} - {lowest_price}")
        else:
            print(f"{model['id']}: No pricing information available.")

## Data for analysis

In [ ]:
# Create a series of test prompts
prompt_list = \
    ["Tell me about yourself?", 
     "Tell me about your C++ skills?", 
     "Have you worked in Machine Learning?",
     "Do you have Python Skills?",
     "Tell me about your python skills?",
     "What is your favorite color?",
     "What is the velocity of an unladen swallow?"]


In [ ]:
%%time

## Test Prompts

import pandas as pd
# prompt_test will be filled with the responses
prompt_test = pd.DataFrame(data=prompt_list,columns=["prompt"])


for model in selected_models:
    print(f"Testing model: {model}")
    rag_system = RAGSystem(model_name=model)
    response = rag_system.query("testing to make sure the prompt engine is working")
    if "difficulties" in response.lower():
        print(f"Model {model} is broken.")
        continue
    prompt_test[model] = [rag_system.query(p) for p in  prompt_test["prompt"]]
    
    
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

prompt_test.to_html(f'prompt_test_{timestamp}.html')